# Visualization notebook

This notebook is meant for visualizing stuff and testing code. 

In [1]:
import numpy as np
import pandas as pd
import os
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import json
from nltk import Tree
from spacy import displacy
import spacy
from dataclasses import dataclass
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
from vectorizer import GrammarVectorizer, make_document
from contextlib import contextmanager
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
@contextmanager
def temp_change_dir(path:str):
    """Changes to a new directory specified by given path, and then reverts back to the old directory"""
    oldpwd = os.getcwd()
    os.chdir(path)
    try:
        yield
    finally:
        os.chdir(oldpwd)
        
def load_pan22() -> pd.DataFrame:
    """Loads PAN22 data as a dataframe"""
    with temp_change_dir("../../"):
        return pd.read_json("data/pan22/preprocessed/pan22_preprocessed.jsonl", lines=True)

In [5]:

pan = load_pan22()

docs = pan["fullText"]
g2v = GrammarVectorizer()
df = g2v.create_vector_df(docs.to_list())
df.insert(1, "author_id", pan["authorIDs"])


#df.to_csv("../../../authorship_analysis/data/document_vectors.csv", index=None)

In [12]:
docs = [
    "This is an extremely wonderful string",
    "The string below me is false.",
    "The string above me is true"
    ]
g2v = GrammarVectorizer()
df = g2v.create_vector_df(docs)
df.shape

(3, 429)